<a href="https://colab.research.google.com/github/kaganseyda/projects/blob/main/project_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'lmsys-chatbot-arena:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F66631%2F8346466%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240702%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240702T092839Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D11b77301ca4bc30a660b859f501c804e94a7f9356a17ddf3d2b92db8b4ff4933ec750586191006875885b94065d4a0adf926c55e6ce1a43f33cdc1dc8bd535811e4fbd64afb9394c6fe8dd3092e0394159ec2d0cc9f67bea667e01084c5b0bb0f64576e807cf4f12323e21c287d244cd9e3d81ae163ba97a7680c5ff334a17ff6ca8777e11f6253abb80a16d48b558985fdab00dfb5908b1b80e815fe1ac4a0a4c5d170177b12950acbb3ff8be1052a35f4af40aa97485f134edd09b06942618c53888dd035cb8f55a95038d7571cc099113ffd7213f67a4c4a27c18c9bff2bfeeed8d4e861d2efa72c488e876ec062ef7ffe97a58749e0d901ff02e8aca4c4e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 59810797 bytes downloaded
Downloaded and uncompressed: lmsys-chatbot-arena
Data source import complete.


In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
train=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
test=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')



In [17]:
train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [50]:
from platform import win32_edition
x = tfidf.fit_transform(train[['prompt', 'response_a', 'response_b', 'model_a', 'model_b']]).toarray()


In [51]:
x_test=tfidf.fit_transform(test[['prompt', 'response_a', 'response_b']]).toarray()

In [52]:
y1=train['winner_model_b']
y2=train['winner_tie']

In [53]:
combined_text_test = test['prompt'] + ' ' + test['response_a'] + ' ' + test['response_b']
X_test = tfidf.fit_transform(combined_text_test)

In [56]:
combined_text_train = train['prompt'] + ' ' + train['response_a'] + ' ' + train['response_b']
X_train = tfidf.fit_transform(combined_text_train)

In [54]:
y_train_model_a = train['winner_model_a']
y_train_model_b = train['winner_model_b']
y_train_tie = train['winner_tie']

In [65]:
tfidf.fit(combined_text_train)
X_train = tfidf.transform(combined_text_train)
X_test = tfidf.transform(combined_text_test)

x = X_train
y = y_train_model_a
model_a = RandomForestClassifier()
model_a.fit(x, y)
pred_model_a = model_a.predict(X_test)

In [59]:
model_a.score(x,y)

0.9993040694538685

In [66]:
x= X_train
y = y_train_model_b
model_b = RandomForestClassifier()
model_b.fit(x, y)
pred_model_b = model_b.predict(X_test)
model_b.score(x,y)

0.9992344763992553

In [67]:
x= X_train
y = y_train_tie
model_tie = RandomForestClassifier()
model_tie.fit(x, y)
pred_model_tie = model_b.predict(X_test)
model_tie.score(x,y)

0.9995302468813613

In [26]:
model_a.score(x,y)

0.9897176261809072

In [62]:
pred_model_a_proba = model_a.predict_proba(X_test)


In [68]:
pred_model_b_proba = model_b.predict_proba(X_test)


In [69]:
pred_model_tie_proba = model_tie.predict_proba(X_test)


In [73]:
pred_model_a_percentage = pred_model_a_proba[:, 1]  # 1. sınıfın olasılığı (winner_model_a)
pred_model_b_percentage = pred_model_b_proba[:, 1]  # 1. sınıfın olasılığı (winner_model_b)
pred_model_tie_percentage = pred_model_tie_proba[:, 1]  # 1. sınıfın olasılığı (winner_tie)

total_percentage = pred_model_a_percentage + pred_model_b_percentage + pred_model_tie_percentage
pred_model_a_percentage /= total_percentage
pred_model_b_percentage /= total_percentage
pred_model_tie_percentage /= total_percentage

# Submission dosyasını hazırlama
submission_df = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': pred_model_a_percentage,
    'winner_model_b': pred_model_b_percentage,
    'winner_tie': pred_model_tie_percentage
})

# Submission dosyasını kaydetme
submission_df.to_csv('submission.csv', index=False)

In [74]:
df=pd.read_csv('submission.csv')
df.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.196296,0.225926,0.577778
1,211333,0.345133,0.353982,0.300885
2,1233961,0.311828,0.419355,0.268817
